In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.0`
import $ivy.`sh.almond::almond-spark:0.10.0`

import $ivy.$                                  

import $ivy.$                               

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}


In [3]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions.avg

import org.apache.spark.sql._

import org.apache.spark.sql.functions.avg

In [7]:
// Could also add SPARK_LOCAL_IP as env var
val spark = SparkSession
            .builder
            .appName("foo1")
            .master("local[*]")
            .config("spark.driver.bindAddress", "127.0.0.1")
            .getOrCreate()

spark: SparkSession = org.apache.spark.sql.SparkSession@37982b85

In [8]:
spark

res7: SparkSession = org.apache.spark.sql.SparkSession@37982b85

In [9]:
val dataDF = spark.createDataFrame(Seq(("Brooke", 20), ("Brooke", 25), 
  ("Denny", 31), ("Jules", 30), ("TD", 35))).toDF("name", "age")

dataDF: DataFrame = [name: string, age: int]

In [10]:
val avgDF = dataDF.groupBy("name").agg(avg("age"))

avgDF: DataFrame = [name: string, avg(age): double]

In [11]:
avgDF.show()

+------+--------+
|  name|avg(age)|
+------+--------+
|Brooke|    22.5|
| Jules|    30.0|
|    TD|    35.0|
| Denny|    31.0|
+------+--------+



In [12]:
val sc = spark.sparkContext

sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@58768acf

In [13]:
val dataRDD = sc.parallelize(Seq(("Brooke", 20.0), ("Denny", 31.0), ("Jules", 30.0), 
  ("TD", 35.0), ("Brooke", 25.0)))

dataRDD: org.apache.spark.rdd.RDD[(String, Double)] = ParallelCollectionRDD[8] at parallelize at cmd12.sc:1

In [14]:
val m = dataRDD.map(f => (f._1, (f._2, 1)))

m: org.apache.spark.rdd.RDD[(String, (Double, Int))] = MapPartitionsRDD[9] at map at cmd13.sc:1

In [15]:
val agesRDD = m.reduceByKey((x, y) => (x._1 + y._1, x._2 + y._2))

agesRDD: org.apache.spark.rdd.RDD[(String, (Double, Int))] = ShuffledRDD[10] at reduceByKey at cmd14.sc:1

In [16]:
m.collect()

res15: Array[(String, (Double, Int))] = Array(
  ("Brooke", (20.0, 1)),
  ("Denny", (31.0, 1)),
  ("Jules", (30.0, 1)),
  ("TD", (35.0, 1)),
  ("Brooke", (25.0, 1))
)

In [17]:
agesRDD.collect()

res16: Array[(String, (Double, Int))] = Array(
  ("Denny", (31.0, 1)),
  ("TD", (35.0, 1)),
  ("Brooke", (45.0, 2)),
  ("Jules", (30.0, 1))
)

In [18]:
val avgRDD = agesRDD.map(f => (f._1, f._2._1/f._2._2))

avgRDD: org.apache.spark.rdd.RDD[(String, Double)] = MapPartitionsRDD[11] at map at cmd17.sc:1

In [19]:
avgRDD.collect()

res18: Array[(String, Double)] = Array(
  ("Denny", 31.0),
  ("TD", 35.0),
  ("Brooke", 22.5),
  ("Jules", 30.0)
)